# --- Day 4: Repose Record ---

## Part One

You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. **Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.**

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

### Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be `10 * 24 = 240`.)

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def read_in():
    with open('../inputs/04/input.txt', 'r') as fp:
        for line in sorted(fp.read().splitlines()):
            yield(parse_line(line))

def parse_line(line):
    dt, event = line.lstrip("[").split("] ")
    return datetime.strptime(dt, "%Y-%m-%d %H:%M"), event

list(read_in())[:10]

[(datetime.datetime(1518, 3, 4, 0, 0), 'Guard #659 begins shift'),
 (datetime.datetime(1518, 3, 4, 0, 39), 'falls asleep'),
 (datetime.datetime(1518, 3, 4, 0, 47), 'wakes up'),
 (datetime.datetime(1518, 3, 5, 0, 4), 'Guard #2161 begins shift'),
 (datetime.datetime(1518, 3, 5, 0, 22), 'falls asleep'),
 (datetime.datetime(1518, 3, 5, 0, 57), 'wakes up'),
 (datetime.datetime(1518, 3, 5, 23, 58), 'Guard #2917 begins shift'),
 (datetime.datetime(1518, 3, 6, 0, 30), 'falls asleep'),
 (datetime.datetime(1518, 3, 6, 0, 51), 'wakes up'),
 (datetime.datetime(1518, 3, 6, 0, 57), 'falls asleep')]

In [2]:
lines = read_in()
sleeps = []
while True:
    try:
        dt, event = next(lines)
    except StopIteration:
        break
    if "begins shift" in event:
        curr_shift = []
        curr_guard = int(event.split()[1].lstrip("#"))
    elif "asleep" in event:
        dt_wakes, _ = next(lines)
        sleeps.append((curr_guard, dt.minute, dt_wakes.minute))

In [3]:
df = (pd.DataFrame(sleeps, columns=["guard", "asleep", "awake"])
      .set_index("guard")
      .assign(length=lambda r: r.awake - r.asleep - 1))
df.head()

,asleep,awake,length
guard,,,
659,39,47,7
2161,22,57,34
2917,30,51,20
2917,57,58,0
2833,33,36,2


In [5]:
guards = df.index.unique().sort_values().values
minute_pivot = pd.DataFrame(0, index=guards, columns=range(60))
for guard in guards:
    for _, row in df.loc[guard].iterrows():
        for minute in np.arange(row.asleep, row.awake):
            minute_pivot.loc[guard, minute] += 1

In [6]:
minute_pivot.values.max()

18

In [7]:
total_sleep = minute_pivot.sum(axis=1).sort_values(ascending=False)
sleepiest_guard = total_sleep.index.values[0]

total_sleep.head()

2917    495
1489    492
659     489
1993    477
1471    350
dtype: int64

In [8]:
sleepiest_minutes = minute_pivot.loc[sleepiest_guard].sort_values(ascending=False)
sleepiest_minute = sleepiest_minutes.index.values[0]

sleepiest_minutes.head()

25    14
29    13
35    13
21    13
22    13
Name: 2917, dtype: int64

In [9]:
sleepiest_guard * sleepiest_minute

72925

### Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)

In [10]:
most_consistent_sleeper = minute_pivot.max(axis=1).idxmax()
most_consistent_minute = minute_pivot.loc[most_consistent_sleeper].idxmax()
(most_consistent_sleeper, most_consistent_minute)

(1489, 33)

In [11]:
most_consistent_sleeper * most_consistent_minute

49137